This notebook use for tunning model using embeddings file and language model embedder

### Check GPU hardware

In [1]:
!nvidia-smi

Mon Mar 14 08:45:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   51C    P8    15W / 170W |     15MiB / 12053MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Import libraries 

In [2]:
# Libraries for system and debug
import sys
import pdb
import os
from datetime import datetime

# Libraries for neural network training
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GRU, LSTM, Bidirectional, Input, Conv1D, Conv2D
from tensorflow.keras.layers import Add, Flatten, subtract, multiply, concatenate
from tensorflow.keras.layers import MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.utils import Sequence
from tensorflow.keras import mixed_precision
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
import tensorflow_addons as tfa
from sklearn.model_selection import KFold, ShuffleSplit
from sklearn.model_selection import train_test_split

from Bio import SeqIO
from bio_embeddings.embed import BeplerEmbedder,ProtTransT5XLU50Embedder,ESM1bEmbedder

# Import accessory modules
import numpy as np
import h5py
import gc
from tqdm import tqdm
import pandas as pd
from pathlib import Path
import shutil

### Set CUDA environment variables

In [3]:
### Setting RAM GPU for training growth 
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2022-03-14 08:45:19.735712: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-14 08:45:22.462892: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8839 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


### Define custom function

In [130]:
def generator_pair(seq_tensor, class_labels, pair_index):
    for index in pair_index:
        yield {"seq1": seq_tensor[seq_index1[index]], "seq2": seq_tensor[seq_index2[index]]}, class_labels[index]

def generator_pair_predict(seq_tensor, pair_index):
    for index in pair_index:
        yield {"seq1": seq_tensor[seq_index1[index]], "seq2": seq_tensor[seq_index2[index]]}

def seq_max(id2seq_file):
    seqs = []
    for line in open(id2seq_file):
        line = line.strip().split('\t')
        seqs.append(len(line[1]))
    
    return max(seqs)
        
def preprocess_embed(id2seq_file, ds_file, e_type):
    id2index = {}
    seqs = []
    index = 0
    sid1_index = 0
    sid2_index = 1
    label_index = 2
    
    for line in open(id2seq_file):
        line = line.strip().split('\t')
        id2index[line[0]] = index
        seqs.append(line[1])
        index += 1

    seq_array = []
    id2_aid = {}
    sid = 0

    max_data = -1
    limit_data = max_data > 0
    raw_data = []
    x = None
    count = 0
    
    # Create sequence array as a list of protein strings
    for line in tqdm(open(ds_file)):
        line = line.rstrip('\n').rstrip('\r').split('\t')
        if id2index.get(line[sid1_index]) is None or id2index.get(line[sid2_index]) is None:
            continue
        if id2_aid.get(line[sid1_index]) is None:
            id2_aid[line[sid1_index]] = sid
            sid += 1
            seq_array.append(seqs[id2index[line[sid1_index]]])
        line[sid1_index] = id2_aid[line[sid1_index]]
        if id2_aid.get(line[sid2_index]) is None:
            id2_aid[line[sid2_index]] = sid
            sid += 1
            seq_array.append(seqs[id2index[line[sid2_index]]])
        line[sid2_index] = id2_aid[line[sid2_index]]
        raw_data.append(line)
        if limit_data:
            count += 1
            if count >= max_data:
                break
    
    # Extract index of 1st and 2nd sequences in pairs
    seq_index1 = np.array([line[sid1_index] for line in tqdm(raw_data)])
    seq_index2 = np.array([line[sid2_index] for line in tqdm(raw_data)])
    

    # Assign labels for pairs of sequences
    class_map = {'0': 1, '1': 0}
    class_labels = np.zeros((len(raw_data), 2))
    for i in range(len(raw_data)):
        class_labels[i][class_map[raw_data[i][label_index]]] = 1
    
    # Pretrained embed
    if e_type == "bepler":
        embedder = BeplerEmbedder()
    elif e_type == "prottrans_t5u50":
        embedder = ProtTransT5XLU50Embedder()
    elif e_type == "esm-1b":
        embedder = ESM1bEmbedder()
        
        
    sequences = pd.read_csv(id2seq_file, sep="\t", header=None)
    sequences = sequences.iloc[:,1].to_list()
        
    embeddings = []
    i = 1
    for sequence in sequences:
        embeddings.append(embedder.embed(sequence))
        if i % 1000 == 0:
            print(i)
        i+=1
        
    embeddings = list(embeddings)
    
    seq_tensor= tf.keras.preprocessing.sequence.pad_sequences(embeddings,  padding='post', 
                                                              dtype='float16', truncating='post', maxlen=seq_max(id2seq_file))
    dim = seq_tensor.shape[2]
    

    Path('preprocess').mkdir(parents=True, exist_ok=True)
    np.savez('preprocess/embed_preprocess_' + e_type + '.npz', 
                 seq_tensor = seq_tensor, seq_index1 = seq_index1, 
                 seq_index2 = seq_index2, class_labels = class_labels, dim = dim)
    
    return seq_tensor, seq_index1, seq_index2, class_labels, dim
        
def leaky_relu(x, alpha = .3):
    return tf.keras.backend.maximum(alpha*x, x)

def build_model(hparams):
    # Input of sequence tensor representations 

    seq_input1 = Input(shape=(seq_size, dim), name='seq1')
    seq_input2 = Input(shape=(seq_size, dim), name='seq2')

    # Define Conv1D and Bi-RNN (GRU/LSTM) use in architecture
    l1=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])
    r1=Bidirectional(GRU(hparams[HP_RNN_HIDDEN_DIM], return_sequences=True))
    l2=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])
    r2=Bidirectional(GRU(hparams[HP_RNN_HIDDEN_DIM], return_sequences=True))
    l3=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])
    r3=Bidirectional(GRU(hparams[HP_RNN_HIDDEN_DIM], return_sequences=True))
    l4=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])
    r4=Bidirectional(GRU(hparams[HP_RNN_HIDDEN_DIM], return_sequences=True))
    l5=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])
    r5=Bidirectional(GRU(hparams[HP_RNN_HIDDEN_DIM], return_sequences=True))
    l6=Conv1D(hparams[HP_CONV_HIDDEN_DIM], hparams[HP_KERNEL_SIZE], activation=hparams[HP_ACTIVATION_CONV], padding=hparams[HP_CONV_PADDING])

    # Siamese architecture

    ### 1st sibling

    # 1st Block RCNN 
    s1=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l1(seq_input1))
    s1=concatenate([r1(s1), s1])

    # 2nd Block RCNN
    s1=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l2(s1))
    s1=concatenate([r2(s1), s1])

    # 3rd Block RCNN
    s1=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l3(s1))
    s1=concatenate([r3(s1), s1])

    # 4th Block RCNN 
    s1=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l4(s1))
    s1=concatenate([r4(s1), s1])

    # 5th Block RCNN
    s1=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l5(s1))
    s1=concatenate([r5(s1), s1])

    # Last convolution
    s1=l6(s1)
    s1=GlobalAveragePooling1D()(s1)

    ### 2nd sibling

    # 1st block RCNN
    s2=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l1(seq_input2))
    s2=concatenate([r1(s2), s2])

    # 2nd block RCNN
    s2=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l2(s2))
    s2=concatenate([r2(s2), s2])

    # 3rd block RCNN
    s2=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l3(s2))
    s2=concatenate([r3(s2), s2])

    # 4th block RCNN
    s2=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l4(s2))
    s2=concatenate([r4(s2), s2])

    # 5th block RCNN
    s2=MaxPooling1D(hparams[HP_POOLING_KERNEL])(l5(s2))
    s2=concatenate([r5(s2), s2])

    # Last convolution
    s2=l6(s2)
    s2=GlobalAveragePooling1D()(s2)

    ### Combine two siblings of siamese architecture
    merge_text = multiply([s1, s2])


    #### MLP Part
    # Set initializer

    # First dense
    x = Dense(hparams[HP_FIRST_DENSE], activation=hparams[HP_ACTIVATION])(merge_text)
    # x = tf.keras.layers.LeakyReLU(alpha=.3)(x)
    x = Dropout(hparams[HP_DROPOUT])(x)

    # Second dense
    x = Dense(int((hparams[HP_CONV_HIDDEN_DIM]+7)/2), activation=hparams[HP_ACTIVATION])(x)
    # x = tf.keras.layers.LeakyReLU(alpha=.3)(x)
    x = Dropout(hparams[HP_DROPOUT])(x)

    # Last softmax
    main_output = Dense(2, activation='softmax')(x)

    # Combine to form functional model
    merge_model = Model(inputs=[seq_input1, seq_input2], outputs=[main_output])
    return merge_model

In [5]:
# ============================================
# Optimisation Flags - Do not remove
# ============================================

# Disables caching (when set to 1) or enables caching (when set to 0) for just-in-time-compilation. When disabled,
# no binary code is added to or retrieved from the cache.
os.environ['CUDA_CACHE_DISABLE'] = '0' # orig is 0

# When set to 1, forces the device driver to ignore any binary code embedded in an application 
# (see Application Compatibility) and to just-in-time compile embedded PTX code instead.
# If a kernel does not have embedded PTX code, it will fail to load. This environment variable can be used to
# validate that PTX code is embedded in an application and that its just-in-time compilation works as expected to guarantee application 
# forward compatibility with future architectures.
os.environ['CUDA_FORCE_PTX_JIT'] = '1'# no orig
os.environ['HOROVOD_GPU_ALLREDUCE'] = 'NCCL'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_GPU_THREAD_COUNT']='1'
os.environ['TF_USE_CUDNN_BATCHNORM_SPATIAL_PERSISTENT'] = '1'
os.environ['TF_ADJUST_HUE_FUSED'] = '1'
os.environ['TF_ADJUST_SATURATION_FUSED'] = '1'
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'
os.environ['TF_SYNC_ON_FINISH'] = '0'
os.environ['TF_AUTOTUNE_THRESHOLD'] = '2'
os.environ['TF_DISABLE_NVTX_RANGES'] = '1'
os.environ["TF_ENABLE_AUTO_MIXED_PRECISION_GRAPH_REWRITE"] = "1"
# =================================================
# mixed_precision.set_global_policy('mixed_float16')

In [6]:
os.chdir('../')
os.getcwd()

'/home/wmbio/WORK/gitworking/PIPR'

In [9]:
# Default hyperparameters
CONV_HIDDEN_DIM = 50
RNN_HIDDEN = 50
N_EPOCHS = 30
HIDDEN_DIM=50
BATCH_SIZE = 32
DTYPE='float16'
LEARNING_RATE=.001
EPSILON=1e-6
MAX_DATASET_SIZE = 11187
DATASET_SIZE = MAX_DATASET_SIZE
KERNEL_SIZE = 3
POOLING_KERNEL = 3
EMBEDDING_TYPE = 'bepler' #bepler, prottrans_t5u50, esm-1b


adam = Adam(learning_rate=LEARNING_RATE, amsgrad=True, epsilon=EPSILON)
get_custom_objects().update({'leaky_relu': leaky_relu})

### Protein EMBEDDING using pLM

In [ ]:
if os.path.isfile('preprocess/embed_preprocess_' + EMBEDDING_TYPE +'.npz'):
    with np.load('preprocess/embed_preprocess_' + EMBEDDING_TYPE + '.npz') as data:
        seq_tensor, seq_index1, seq_index2, class_labels, dim = data['seq_tensor'], data['seq_index1'], data['seq_index2'], data['class_labels'], data['dim']
else :
    seq_tensor, seq_index1, seq_index2, class_labels, dim = preprocess_embed(id2seq_file='data/wmbio_set/Train_set/human_custom_seq.tsv',
                                                        ds_file='data/wmbio_set/Train_set/human_custom_ppi_pair.tsv',
                                                        e_type=EMBEDDING_TYPE)

# max sequence length
seq_size = seq_max('data/wmbio_set/Train_set/human_custom_seq.tsv')

### Define callbacks for monitor

In [ ]:
### Learning rate schedule for optimization during training
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.4,
    patience=4,
    verbose=0,
    mode="auto",
    min_lr=1e-5)

# Schedule early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=6,
    mode='min',
    restore_best_weights=True)

final_reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="loss",
    factor=0.4,
    patience=4,
    verbose=0,
    mode="auto",
    min_lr=1e-5)

final_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', 
    verbose=1,
    patience=7,
    mode='max',
    restore_best_weights=True)

### Define performance metrics

In [ ]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      tfa.metrics.MatthewsCorrelationCoefficient(num_classes=2, name='mcc'),
      tfa.metrics.F1Score(num_classes=2, threshold=0.5, name='f1-score'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

### Summary of model architecture - default parameter

In [ ]:
HP_EPSILON = hp.HParam('epsilon', hp.Discrete([1e-6]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-3]))
HP_FIRST_DENSE = hp.HParam('first_dense', hp.Discrete([100]))
HP_KERNEL_SIZE = hp.HParam('kernel_size', hp.Discrete([3]))
HP_POOLING_KERNEL = hp.HParam('pooling_kernel', hp.Discrete([3]))
HP_CONV_HIDDEN_DIM = hp.HParam('conv_hidden_dim', hp.Discrete([50]))
HP_RNN_HIDDEN_DIM = hp.HParam('rnn_hidden_dim', hp.Discrete([50]))
HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['leaky_relu']))
HP_ACTIVATION_CONV = hp.HParam('activation_conv', hp.Discrete(['linear']))
HP_REGULARIZER = hp.HParam('regularizer', hp.Discrete([0]))
HP_CONV_PADDING = hp.HParam('conv_padding', hp.Discrete(['valid']))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0e-1]))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([256]))
HP_LEAKY_RELU = hp.HParam('leaky_relu', hp.Discrete([3e-1]))
METRIC_ACCURACY = 'accuracy'

hparams = {
  HP_EPSILON: EPSILON,
  HP_LEARNING_RATE: LEARNING_RATE,
  HP_FIRST_DENSE: 100,
  HP_KERNEL_SIZE: 3,
  HP_POOLING_KERNEL: 3,
  HP_CONV_HIDDEN_DIM: 50,
  HP_RNN_HIDDEN_DIM: 50,
  HP_ACTIVATION: 'leaky_relu',
  HP_ACTIVATION_CONV: 'relu',
  HP_REGULARIZER: 0,
  HP_CONV_PADDING: 'valid',
  HP_DROPOUT: 3e-1,
  HP_BATCH_SIZE: 256,
  HP_LEAKY_RELU: 3e-1
}

model = build_model(hparams)
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

### Loop over all configurations

In [ ]:
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(n_splits=5, test_size=2, random_state=331)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cnt = 0

# save hparams
SAVE_MODEL = 'save_model_pretrained_' + EMBEDDING_TYPE
Path(SAVE_MODEL).mkdir(parents=True, exist_ok=True)

for train, test in kf.split(class_labels):
    cnt+=1
    merge_model = None
    with tf.device('/cpu:0'): # GPU mode일 경우, remove
        merge_model = build_model(hparams)  
        tf.keras.utils.plot_model(merge_model, to_file=SAVE_MODEL + 'model.png', show_shapes=True)


        merge_model.compile(optimizer=Adam(learning_rate=hparams[HP_LEARNING_RATE], amsgrad=True, epsilon=hparams[HP_EPSILON]), 
                          loss='categorical_crossentropy', metrics=METRICS)

        # Create train
        # from generator
        train_dataset = tf.data.Dataset.from_generator(generator_pair, 
                                                       args=[seq_tensor, class_labels, train], 
                                                       output_types=({"seq1": DTYPE, "seq2": DTYPE}, DTYPE), 
                                                       output_shapes = ({"seq1": (seq_size, dim), "seq2": (seq_size, dim)}, (2,)) )
        train_dataset = train_dataset.shuffle(1024).repeat(N_EPOCHS).batch(hparams[HP_BATCH_SIZE])
        train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

        # Create test
        test_dataset = tf.data.Dataset.from_generator(generator_pair, args=[seq_tensor, class_labels, test], 
                                                      output_types=({"seq1": DTYPE, "seq2": DTYPE}, DTYPE), 
                                                      output_shapes = ({"seq1": (seq_size, dim), "seq2": (seq_size, dim)}, (2,)) )
        test_dataset = test_dataset.batch(hparams[HP_BATCH_SIZE])

        # Save the best model base on val_accuracy
        checkpoint = ModelCheckpoint(filepath=SAVE_MODEL + str(cnt)+'-fold_best_model.hdf5', 
                                     monitor='val_accuracy',verbose=1, save_best_only=True, mode='max')

        # Fit model
        print(f'==================== Training time =====================')
        history_model = merge_model.fit(train_dataset, 
                                        epochs=N_EPOCHS, 
                                        steps_per_epoch=len(train) // 128, 
                                        validation_data=test_dataset,
                                        callbacks=[checkpoint, reduce_lr, early_stopping,                                               
                                                  tf.keras.callbacks.CSVLogger(SAVE_MODEL + 'history.csv')])
    # file rename
    shutil.move(SAVE_MODEL + 'history.csv', SAVE_MODEL + str(cnt) + '-fold_history.csv')

## Final Modeling

In [ ]:
merge_model = None
merge_model = build_model(hparams)  

# ADAM
merge_model.compile(optimizer=Adam(learning_rate=hparams[HP_LEARNING_RATE], 
                                   amsgrad=True, epsilon=hparams[HP_EPSILON]), 
                    loss='categorical_crossentropy', metrics=METRICS)

# Create train
train_dataset = tf.data.Dataset.from_generator(generator_pair, 
                                               args=[seq_tensor, class_labels, train], 
                                               output_types=({"seq1": DTYPE, "seq2": DTYPE}, DTYPE), 
                                               output_shapes = ({"seq1": (seq_size, dim), "seq2": (seq_size, dim)}, (2,)) )
train_dataset = train_dataset.shuffle(1024).repeat(N_EPOCHS).batch(hparams[HP_BATCH_SIZE])
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

# Fit model
history = merge_model.fit(train_dataset, 
                epochs=N_EPOCHS,
                steps_per_epoch=len(seq_tensor) // 128, 
                callbacks=[final_reduce_lr, final_early_stopping])

# model save
SAVE_MODEL = 'final_model/'
Path(SAVE_MODEL).mkdir(parents=True, exist_ok=True)
merge_model.save(SAVE_MODEL + 'PIPR_'+EMBEDDING_TYPE+'_final.h5')

* **predict**

In [142]:
seq_tensor, seq_index1, seq_index2, class_labels, dim = preprocess_embed(id2seq_file='data/wmbio_set/Test_set/human_test_seq.tsv',
                                                                         ds_file='data/wmbio_set/Test_set/human_test_pair.tsv',
                                                                         e_type=EMBEDDING_TYPE,
                                                                         predict=True) 

# max sequence length
seq_size = seq_max('data/wmbio_set/Train_set/human_custom_seq.tsv')

5it [00:00, 18078.90it/s]
100%|██████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 295373.52it/s]


In [114]:
model = tf.keras.models.load_model('final_model/PIPR_befler_final.h5')

In [ ]:
model.predict([seq_tensor[seq_index1], seq_tensor[seq_index2]])

array([[0.82994366, 0.17005637],
       [0.99306023, 0.00693977],
       [0.81467044, 0.18532953],
       [0.00521381, 0.99478614],
       [0.9263858 , 0.07361418]], dtype=float32)